In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load dataset
df = pd.read_csv("../data/processed_experiment_results.csv")

# Overview of Data
print("First 5 rows:")
print(df.head())

print("\n Summary Statistics:")
print(df.describe())